In [1]:
import numpy as np
import scipy.stats as st
import math
import matplotlib.pyplot as plt
import matplotlib

In [2]:
plt.rcParams.update({'legend.fontsize': 30, 'axes.labelsize' : 30, 'axes.titlesize': 20, 'xtick.labelsize' : 25, 'ytick.labelsize' : 25})

Приступим к практическому внедрению метода градиентного спуска в жизнь (и в нашу задачу, в частности).

In [3]:
N = 10**3
TernConst = 3
grad_error = 10**(-10)
point_error = 10**(-10)
step = 10

Тут представлен тернарный поиск с количеством шагов $TernConst$.

In [4]:
def _ternarnik(x, y, func):
    m1 = x + (y - x) / 3
    m2 = y - (y - x) / 3
    if func(m1) < func(m2):
        return (m1, y)
    else:
        return (x, m2)
    
def do_ternar(x, y, func):
    for i in range(TernConst):
        (x, y) = _ternarnik(x, y, func)
    return (x, y)

Эта операция служит для получения случайных значений массивов $p$ и $\pi$. Получившиеся значения $p \in [0, 2]$.

In [5]:
def make_random(m):
    n = np.random.randint(1, 100)
    P_base = list(np.random.rand(n - 1))
    P_base.append(0)
    P_base.append(1)
    P_base.sort()
    P = np.array([P_base[i] - P_base[i - 1] for i in range(1, n + 1)])
    P = np.floor(P * 100) / 100
    P[0] += 1 - np.sum(P)
    p = np.floor(np.random.rand(n, m) * 2 * 100) / 100
    return [P, p]

Здесь описаны функции получения значений $F$ и $\nabla F$ (по уже полученной формуле и беря проекцию под конец).

In [6]:
def base_func(P, p, x):
    x = np.array(x)
    p = np.array(p)
    result = 0
    for (Pi, pi) in zip (P, p):
        result += Pi * math.log(pi @ x)
    return result

def base_grad(P, p, x):
    n = len(x)
    x = np.array(x)
    p = np.array(p)
    result = np.zeros(n)
    for (Pi, pi) in zip (P, p):
        result += pi * (Pi / (pi @ x))
    result -= np.ones((n)) * ((np.ones((n)) @ result) / n)
    return result

In [7]:
def make_func(P, p):
    return lambda x: base_func(P, p, x)

def make_grad(P, p):
    return lambda x: base_grad(P, p, x)

Следующая точка градиентного поиска получается из этих функций. Первую из них, с заданным шагом, мы пока не используем. Во второй, мы сначала вычисляем предельную точку множества по направлению градиента (которая выйдет за $x \succeq 0$). А затем отправляем $x_{i - 1}$ и $x_{пред}$ на тернарный поиск и возвращаем среднее от результата.

In [8]:
def next_grad_point_with_step(x_prev, lambd, grad):
    return(x_prev + lambd * grad(x_prev))

def next_grad_point(x_prev, grad, func):
    grad = grad(x_prev)
    t = None
    for (x, g) in zip(x_prev, grad):
        comp = None
        if g < 0:
            comp = -x / g
            if (t == None) or comp < t:
                t = comp
    if t <= step:
        points = do_ternar(x_prev, x_prev + t * 98 / 100 * grad, func)
    else:
        points = do_ternar(x_prev, x_prev + step * grad, func)
    return((points[0] + points[1]) / 2)

Конфиг наших экспериментов. Состоит из задаваемой части и случайной, в которой мы только задаем размерность.

In [9]:
config_base = [
    {
        'P': [0.5, 0.5],
        'p': [[1, 1], [1, 1]], 
        'lambda': 1,
        'Graphs': []
    },
    {
        'P': [0.5, 0.5],
        'p': [[1, 20], [1, 1]],
        'lambda': 1,
        'Graphs': []
    },
    {
        'P': [0.5, 0.5],
        'p': [[19, 20], [1, 1]], 
        'lambda': 1,
        'Graphs': []
    },
    {
        'P': [0.1, 0.9],
        'p': [[19, 1], [20, 1]], 
        'lambda': 1,
        'Graphs': []
    },
    {
        'P':  [0.0133953, 0.02577793, 0.18287636, 0.16707962, 0.09537904, 0.2392396, 0.04887821, 0.01878279, 0.14279949, 0.06579168],
        'p': np.array([[5.9281099,  7.45136227, 5.73222386, 7.58610628, 5.71057664, 5.96841042,
  7.48860871, 5.58605544, 5.25149058, 6.33524978],
 [6.13194949, 5.23094632, 5.63525985, 5.95314311, 5.6840923,  6.06803439, 7.93416354, 7.10238094, 7.55517211, 6.85518542]]).T, 
        'lambda': 1,
        'Graphs': []
    }
]
random_config = [3, 3, 3, 4, 4, 4, 4, 6, 6, 6, 10, 10, 20]

In [10]:
data = config_base.copy()
for k in random_config:
    ran = make_random(k)
    data.append({
        'P': ran[0],
        'p': ran[1],
        'lambda': 1,
        'Graphs': []
    })

Следующии три окошка представляют из себя три серии экспериментов, с различным выбором начальных точек:
  * $x_0 = (\frac{1}{n}, \frac{1}{n},..., \frac{1}{n})$
  * $x_0 = (1, 0,..., 0)$
  * $x_0 = (\frac{1}{2}, \frac{1}{2}, 0,..., 0)$  
  Каждая серия заканчивается либо при уменьшении нормы градиента или нормы разности $x_i - x_{i - 1}$ ниже определенного значения, либо по достижении предельного количества шагов.

In [11]:
for data_map in data:
    P = data_map['P']
    p = data_map['p']
    lambd = data_map['lambda'] / 10
    func = make_func(P, p)
    grad = make_grad(P, p)
    x0 = np.ones((len(p[0]))) / len(p[0])
    values = [func(x0)]
    Xs = [x0]
    data_map['Graphs'].append({
        'x0': x0,
        'values': values,
        'Xs': Xs
    })
    for i in range(N):
        if np.linalg.norm(grad(Xs[-1])) < grad_error:
            break
        Xs.append(next_grad_point(Xs[-1], grad, func))
        values.append(func(Xs[-1]))
        if (np.linalg.norm(Xs[-1] - Xs[-2]) < point_error):
            break

In [12]:
for data_map in data:
    P = data_map['P']
    p = data_map['p']
    lambd = data_map['lambda'] / 10
    func = make_func(P, p)
    grad = make_grad(P, p) 
    x0 = np.zeros(len(p[0]))
    x0[0] = 0.99
    x0[1] = 0.01
    values = [func(x0)]
    Xs = [x0]
    data_map['Graphs'].append({
        'x0': x0,
        'values': values,
        'Xs': Xs
    })
    for i in range(N):
        if np.linalg.norm(grad(Xs[-1])) < grad_error:
            break
        Xs.append(next_grad_point(Xs[-1], grad, func))
        values.append(func(Xs[-1]))
        if (np.linalg.norm(Xs[-1] - Xs[-2]) < point_error):
            break

In [13]:
for data_map in data:
    P = data_map['P']
    p = data_map['p']
    lambd = data_map['lambda'] / 10
    func = make_func(P, p)
    grad = make_grad(P, p) 
    x0 = np.zeros(len(p[0]))
    x0[0] = 0.5
    x0[1] = 0.5
    values = [func(x0)]
    Xs = [x0]
    data_map['Graphs'].append({
        'x0': x0,
        'values': values,
        'Xs': Xs
    })
    for i in range(N):
        if np.linalg.norm(grad(Xs[-1])) < grad_error:
            break
        Xs.append(next_grad_point(Xs[-1], grad, func))
        values.append(func(Xs[-1]))
        if (np.linalg.norm(Xs[-1] - Xs[-2]) < point_error):
            break

Наконец, сравнительные графики для каждого набора $p$ и $\pi$ для всех трех серий

In [14]:
rows_per_data = 2 + (len(data[0]['Graphs']) // 3 + 1) * 2 + 1
gridsize = (len(data) * rows_per_data, 3)
fig = plt.figure(figsize=(40, 40 * len(data)), facecolor=matplotlib.colors.get_named_colors_mapping()['xkcd:cream'])
plt.subplots_adjust(hspace=0.8)
for i in range(len(data)):
    data_map = data[i]
    P = data_map['P']
    p = data_map['p']
    labels = ['$x_{' + '{}'.format(i + 1) + '}$' for i in range(len(P))]
    ax0 = plt.subplot2grid(gridsize, (i * rows_per_data, 0), colspan=3, rowspan=2)
    params = ''
    if len(p) < 15:
        for (Pi, pi) in zip(P, p):
            params += r'$\pi$: {:.2f}, $p_j$: {}'.format(Pi, pi) + '\n'
    else:
        params = "Size of p: ({}, {})".format(len(p), len(p[0]))
    ax0.set_title(params, {'fontsize': 30}, loc='left')
    ax0.set_xlabel(r'$k$')
    ax0.set_ylabel(r'$F(x_k)$')
    graphs = data_map['Graphs']
    for j in range(len(graphs)):
        graph = graphs[j]
        Xs = np.array(graph['Xs'])
        x0 = graph['x0']
        n = len(Xs)
        graph_x = range(n)
        ax0.plot(graph_x, graph['values'], label='$x_0 = ${}'.format(np.floor(x0*100) / 100))
        axi = plt.subplot2grid(gridsize, (i * rows_per_data + (j // 3) * 2 + 2, j % 3), colspan=1, rowspan=2)
        params = ''
        axi.stackplot(graph_x, *[Xs[:, i] for i in range(len(p[0]))], labels=labels)
        axi.legend()
        axi.set_title('$x_0 = ${}'.format(np.floor(x0*100) / 100))
        axi.set_xlabel(r'$k$')
        axi.set_ylabel(r'$X$')
    ax0.legend()

In [15]:
import pprint
pprint.pprint(data)

[{'Graphs': [{'Xs': [array([0.5, 0.5])],
              'values': [0.0],
              'x0': array([0.5, 0.5])},
             {'Xs': [array([0.99, 0.01])],
              'values': [0.0],
              'x0': array([0.99, 0.01])},
             {'Xs': [array([0.5, 0.5])],
              'values': [0.0],
              'x0': array([0.5, 0.5])}],
  'P': [0.5, 0.5],
  'lambda': 1,
  'p': [[1, 1], [1, 1]]},
 {'Graphs': [{'Xs': [array([0.5, 0.5]),
                     array([0.08259259, 0.91740741]),
                     array([0.01364307, 0.98635693]),
                     array([0.00225363, 0.99774637]),
                     array([3.72266866e-04, 9.99627733e-01]),
                     array([6.14929711e-05, 9.99938507e-01]),
                     array([1.01577278e-05, 9.99989842e-01]),
                     array([1.67790615e-06, 9.99998322e-01]),
                     array([2.77165238e-07, 9.99999723e-01]),
                     array([4.57835912e-08, 9.99999954e-01]),
                     arra

                     array([0.0220057, 0.9779943]),
                     array([0.0220057, 0.9779943]),
                     array([0.0220057, 0.9779943]),
                     array([0.0220057, 0.9779943]),
                     array([0.0220057, 0.9779943]),
                     array([0.0220057, 0.9779943]),
                     array([0.02200569, 0.97799431]),
                     array([0.02200569, 0.97799431]),
                     array([0.02200569, 0.97799431]),
                     array([0.02200569, 0.97799431]),
                     array([0.02200569, 0.97799431]),
                     array([0.02200569, 0.97799431]),
                     array([0.02200569, 0.97799431]),
                     array([0.02200569, 0.97799431]),
                     array([0.02200569, 0.97799431]),
                     array([0.02200569, 0.97799431]),
                     array([0.02200569, 0.97799431]),
                     array([0.02200569, 0.97799431]),
                     array([0.02200569, 

 {'Graphs': [{'Xs': [array([0.33333333, 0.33333333, 0.33333333]),
                     array([0.61227002, 0.05506173, 0.33266826]),
                     array([0.73247207, 0.0210866 , 0.24644133]),
                     array([0.70723494, 0.08210336, 0.2106617 ]),
                     array([0.73065708, 0.04038269, 0.22896024]),
                     array([0.72549509, 0.05326065, 0.22124426]),
                     array([0.72986145, 0.04552799, 0.22461056]),
                     array([0.72987867, 0.04584454, 0.22427678]),
                     array([0.73014249, 0.04540575, 0.22445176]),
                     array([0.72978829, 0.04604752, 0.22416419]),
                     array([0.73030691, 0.04511347, 0.22457963]),
                     array([0.72955064, 0.04647538, 0.22397398]),
                     array([0.73064995, 0.04449408, 0.22485596]),
                     array([0.72904351, 0.04738583, 0.22357066]),
                     array([0.7313729 , 0.04318517, 0.22544193]),
          

                     array([0.7320297 , 0.04197814, 0.22599217]),
                     array([0.72693632, 0.05114451, 0.22191917]),
                     array([0.73104468, 0.04371909, 0.22523623]),
                     array([0.72840137, 0.04852917, 0.22306946]),
                     array([0.73226724, 0.04155852, 0.22617424]),
                     array([0.72657769, 0.05178226, 0.22164005]),
                     array([0.73073523, 0.04426425, 0.22500052]),
                     array([0.72885261, 0.04772008, 0.22342731]),
                     array([0.73163574, 0.04270727, 0.22565699]),
                     array([0.72755499, 0.05004562, 0.22239939]),
                     array([0.73157748, 0.04277974, 0.22564278]),
                     array([0.72761549, 0.04993439, 0.22245012]),
                     array([0.73163055, 0.04268466, 0.22568479]),
                     array([0.72753532, 0.05007731, 0.22238737]),
                     array([0.73156142, 0.04280683, 0.22563175]),
          

                     array([0.73133074, 0.04321438, 0.22545488]),
                     array([0.72798043, 0.04928241, 0.22273716]),
                     array([0.73194546, 0.04212733, 0.22592721]),
                     array([0.72706332, 0.05091848, 0.2220182 ]),
                     array([0.73115425, 0.04352587, 0.22531988]),
                     array([0.72824059, 0.04881705, 0.22294236]),
                     array([0.73217002, 0.04172954, 0.22610045]),
                     array([0.7267241 , 0.05152194, 0.22175396]),
                     array([0.73086159, 0.04404172, 0.2250967 ]),
                     array([0.72866885, 0.04804978, 0.22328137]),
                     array([0.73189396, 0.042238  , 0.22586804]),
                     array([0.72715769, 0.05075252, 0.22208979]),
                     array([0.7312352, 0.043384 , 0.2253808]),
                     array([0.72812226, 0.04902888, 0.22284886]),
                     array([0.73206783, 0.04191061, 0.22602156]),
             

...

                         -0.022079485352974073,
                         -0.022079485352974007,
                         -0.022079485352974427,
                         -0.02207948535297419,
                         -0.02207948535297432,
                         -0.02207948535297398,
                         -0.022079485352973324,
                         -0.022079485352971856,
                         -0.022079485352971485,
                         -0.02207948535297176,
                         -0.022079485352971964,
                         -0.02207948535297084,
                         -0.022079485352970114,
                         -0.02207948535297075,
                         -0.022079485352971072,
                         -0.02207948535297133,
                         -0.022079485352971488,
                         -0.022079485352972075,
                         -0.02207948535297254,
                         -0.02207948535297008,
                         -0.022079485352969993,
 

                     array([0.02847604, 0.52131523, 0.02400164, 0.42620709]),
                     array([0.02847605, 0.52131522, 0.02400164, 0.42620709]),
                     array([0.02847604, 0.52131523, 0.02400164, 0.42620709]),
                     array([0.02847606, 0.52131522, 0.02400164, 0.42620708]),
                     array([0.02847603, 0.52131523, 0.02400165, 0.42620709]),
                     array([0.02847605, 0.52131522, 0.02400164, 0.42620708]),
                     array([0.02847604, 0.52131523, 0.02400164, 0.42620709]),
                     array([0.02847605, 0.52131522, 0.02400164, 0.42620708]),
                     array([0.02847604, 0.52131523, 0.02400164, 0.42620709]),
                     array([0.02847607, 0.52131521, 0.02400164, 0.42620707]),
                     array([0.02847602, 0.52131523, 0.02400165, 0.4262071 ]),
                     array([0.02847605, 0.52131522, 0.02400164, 0.42620708]),
                     array([0.02847603, 0.52131523, 0.02400164, 

                         -0.022079485352981498,
                         -0.022079485352982015,
                         -0.022079485352979614,
                         -0.022079485352979863,
                         -0.022079485352979763,
                         -0.022079485352979614,
                         -0.0220794853529797,
                         -0.022079485352979787,
                         -0.022079485352979433,
                         -0.02207948535297871,
                         -0.022079485352978673,
                         -0.022079485352978035,
                         -0.02207948535297804,
                         -0.022079485352978122,
                         -0.022079485352978438,
                         -0.022079485352978708,
                         -0.02207948535297921,
                         -0.02207948535297929,
                         -0.022079485352975894,
                         -0.022079485352976255,
                         -0.022079485352976814

                     array([0.02847603, 0.52131523, 0.02400165, 0.4262071 ]),
                     array([0.02847605, 0.52131522, 0.02400164, 0.42620708]),
                     array([0.02847604, 0.52131523, 0.02400164, 0.42620709]),
                     array([0.02847605, 0.52131522, 0.02400164, 0.42620708]),
                     array([0.02847603, 0.52131523, 0.02400164, 0.42620709]),
                     array([0.02847607, 0.52131521, 0.02400164, 0.42620707]),
                     array([0.02847602, 0.52131523, 0.02400165, 0.4262071 ]),
                     array([0.02847605, 0.52131522, 0.02400164, 0.42620708]),
                     array([0.02847603, 0.52131523, 0.02400164, 0.42620709]),
                     array([0.02847605, 0.52131522, 0.02400164, 0.42620708]),
                     array([0.02847604, 0.52131523, 0.02400164, 0.42620709]),
                     array([0.02847604, 0.52131523, 0.02400164, 0.42620709]),
                     array([0.02847602, 0.52131524, 0.02400165, 

                         -0.022079485352861895,
                         -0.0220794853528624,
                         -0.022079485352862728,
                         -0.022079485352862825,
                         -0.02207948535286282,
                         -0.022079485352861156,
                         -0.022079485352860587,
                         -0.02207948535285976,
                         -0.022079485352859262,
                         -0.022079485352859106,
                         -0.022079485352859016,
                         -0.022079485352858436,
                         -0.022079485352858343,
                         -0.022079485352858693,
                         -0.022079485352858495,
                         -0.022079485352858905,
                         -0.02207948535285886,
                         -0.022079485352858433,
                         -0.022079485352858665,
                         -0.022079485352858634,
                         -0.02207948535285834